In [1]:
# Load libraries
import numpy as np
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.losses import MeanAbsoluteError
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.optimizers import Adam

In [2]:
data = pd.read_csv('IS_data.csv')
data

,Company,Time,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,CTAS,2019-05-01,6892.3,1147.9,885.0,64.97,8.25,6.17,2.05,1284.4,1147.9,654.6,221.83
1,PGR,2014-12-01,19377.3,2029.1,1281.0,32.81,2.17,2.02,0.69,2029.1,2029.1,1195.1,26.99
2,ELP,2018-12-01,3857.8,583.3,363.5,14.10,1.33,1.12,0.37,676.6,583.3,306.2,7.83
3,PHM,2016-12-01,7676.5,980.2,602.7,22.59,1.76,1.82,0.36,994.0,980.2,619.1,18.38
4,F,2019-12-01,155900.0,2658.0,47.0,39.25,0.01,0.40,0.60,2658.0,2658.0,1600.5,9.30
5,WM,2011-12-01,13378.0,2081.0,961.0,28.48,2.05,1.98,1.36,2132.0,2081.0,929.5,32.71
6,SIRI,2019-12-01,7794.0,1756.0,914.0,1.73,0.20,0.19,0.05,1897.0,1756.0,851.9,7.15
7,SCCO,2017-12-01,6654.5,2618.9,728.5,8.61,0.94,1.88,0.59,2624.8,2618.9,1449.7,46.81
8,BAX,2017-12-01,10584.0,1510.0,602.0,19.49,1.11,1.59,0.61,1664.0,1510.0,865.6,64.64
9,CTSH,2014-12-01,10263.0,1885.0,1439.0,16.88,2.37,1.98,0.00,1921.0,1885.0,1201.3,52.66


In [3]:
prices = data['Price'] 
data = data.drop(['Company', 'Time', 'Price'], axis = 1)
data

,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income
0,6892.3,1147.9,885.0,64.97,8.25,6.17,2.05,1284.4,1147.9,654.6
1,19377.3,2029.1,1281.0,32.81,2.17,2.02,0.69,2029.1,2029.1,1195.1
2,3857.8,583.3,363.5,14.10,1.33,1.12,0.37,676.6,583.3,306.2
3,7676.5,980.2,602.7,22.59,1.76,1.82,0.36,994.0,980.2,619.1
4,155900.0,2658.0,47.0,39.25,0.01,0.40,0.60,2658.0,2658.0,1600.5
5,13378.0,2081.0,961.0,28.48,2.05,1.98,1.36,2132.0,2081.0,929.5
6,7794.0,1756.0,914.0,1.73,0.20,0.19,0.05,1897.0,1756.0,851.9
7,6654.5,2618.9,728.5,8.61,0.94,1.88,0.59,2624.8,2618.9,1449.7
8,10584.0,1510.0,602.0,19.49,1.11,1.59,0.61,1664.0,1510.0,865.6
9,10263.0,1885.0,1439.0,16.88,2.37,1.98,0.00,1921.0,1885.0,1201.3


In [4]:
col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")

# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Price', prices)
data

Total Revenues
Done
Operating Income
Done
Net Income
Done
Revenue Per Share
Done
Basic EPS
Done
Normalized Basic EPS
Done
Dividend Per Share
Done
EBITA
Done
EBIT
Done
Normalized Net Income
Done


,Total Revenues,Operating Income,Net Income,Revenue Per Share,Basic EPS,Normalized Basic EPS,Dividend Per Share,EBITA,EBIT,Normalized Net Income,Price
0,0.013132,0.152721,0.191876,0.000414,0.000911,0.000826,0.155303,0.151308,0.152721,0.151562,221.83
1,0.036961,0.159877,0.195145,0.000208,0.000790,0.000720,0.052273,0.157298,0.159877,0.158824,26.99
2,0.007341,0.148136,0.187570,0.000089,0.000773,0.000697,0.028030,0.146419,0.148136,0.146882,7.83
3,0.014629,0.151359,0.189545,0.000143,0.000781,0.000715,0.027273,0.148972,0.151359,0.151086,18.38
4,0.297524,0.164984,0.184957,0.000249,0.000746,0.000678,0.045455,0.162357,0.164984,0.164270,9.30
5,0.025511,0.160299,0.192503,0.000181,0.000787,0.000719,0.103030,0.158126,0.160299,0.155255,32.71
6,0.014853,0.157659,0.192115,0.000010,0.000750,0.000673,0.003788,0.156236,0.157659,0.154213,7.15
7,0.012679,0.164667,0.190584,0.000054,0.000765,0.000716,0.044697,0.162090,0.164667,0.162244,46.81
8,0.020178,0.155661,0.189539,0.000123,0.000768,0.000709,0.046212,0.154362,0.155661,0.154397,64.64
9,0.019566,0.158707,0.196450,0.000107,0.000794,0.000719,0.000000,0.156429,0.158707,0.158907,52.66


In [5]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:10] # Parameters
y = NpMatrix[:,10] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))
number_of_features = len(X[1])

X: [[0.01313249 0.15272091 0.19187598 ... 0.15130826 0.15272091 0.15156246]
 [0.03696098 0.15987705 0.19514547 ... 0.15729821 0.15987705 0.1588236 ]
 [0.00734093 0.14813585 0.18757033 ... 0.14641947 0.14813585 0.14688201]
 ...
 [0.31257509 0.34565004 0.29170268 ... 0.38301227 0.34565004 0.30593264]
 [0.00518902 0.15225639 0.19026601 ... 0.14998432 0.15225639 0.15134751]
 [0.00816982 0.15014171 0.18915389 ... 0.147815   0.15014171 0.14926791]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 10
Total Number of Training instances: 5000


[221.83  26.99   7.83 ...  42.53  50.32  32.35]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 221.83
Total number of y values 5000


In [6]:
#Loss functions
# https://keras.io/api/losses/regression_losses/#meanabsoluteerror-class
# https://keras.io/api/losses/

# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=10, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))
    
    network.add(layers.Dense(units=10, activation='relu', input_shape=(number_of_features,)))
    
    #Add Dropout Layer
    network.add(layers.Dropout(0.3))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=1, activation='relu'))

    # Compile neural network
    network.compile(#loss='mse', # Mean Squared Error
                    loss = MeanAbsoluteError(),
                    optimizer='adam', # Root Mean Square Propagation
                    metrics=['mae', 'mse']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [7]:
neural_network = KerasRegressor(build_fn=create_network, 
                                 epochs= 1000, 
                                 batch_size=5)

In [8]:
# Evaluate neural network using three-fold cross-validation
scores = cross_val_score(neural_network, X, y, cv=5)

Train on 4000 samples
Epoch 1/1000
4000/4000 [==============================] - 3s 729us/sample - loss: 59.7252 - mae: 59.7252 - mse: 23503.1699
Epoch 2/1000
4000/4000 [==============================] - 1s 364us/sample - loss: 50.4940 - mae: 50.4939 - mse: 21507.4453
Epoch 3/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 50.4153 - mae: 50.4153 - mse: 21474.8887
Epoch 4/1000
4000/4000 [==============================] - 2s 378us/sample - loss: 50.0486 - mae: 50.0486 - mse: 21571.6445
Epoch 5/1000
4000/4000 [==============================] - 2s 393us/sample - loss: 49.8723 - mae: 49.8723 - mse: 21615.9199
Epoch 6/1000
4000/4000 [==============================] - 2s 401us/sample - loss: 49.1808 - mae: 49.1808 - mse: 21362.5957
Epoch 7/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 49.3696 - mae: 49.3696 - mse: 21423.1484
Epoch 8/1000
4000/4000 [==============================] - 2s 414us/sample - loss: 48.8722 - mae: 48.8722 - mse: 21358

4000/4000 [==============================] - 2s 386us/sample - loss: 45.3466 - mae: 45.3466 - mse: 21262.8223
Epoch 67/1000
4000/4000 [==============================] - 1s 360us/sample - loss: 45.2637 - mae: 45.2637 - mse: 21232.3164
Epoch 68/1000
4000/4000 [==============================] - 2s 412us/sample - loss: 45.2791 - mae: 45.2791 - mse: 21189.6621
Epoch 69/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 45.4962 - mae: 45.4963 - mse: 21233.3301
Epoch 70/1000
4000/4000 [==============================] - 2s 388us/sample - loss: 44.9053 - mae: 44.9053 - mse: 21160.7949
Epoch 71/1000
4000/4000 [==============================] - 2s 380us/sample - loss: 45.5838 - mae: 45.5838 - mse: 21322.1289
Epoch 72/1000
4000/4000 [==============================] - 2s 389us/sample - loss: 45.4320 - mae: 45.4320 - mse: 21289.3457
Epoch 73/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 45.3729 - mae: 45.3729 - mse: 21175.1055
Epoch 74/1000
4000/400

4000/4000 [==============================] - 1s 342us/sample - loss: 44.6087 - mae: 44.6086 - mse: 21069.6191
Epoch 132/1000
4000/4000 [==============================] - 1s 363us/sample - loss: 45.1707 - mae: 45.1707 - mse: 21254.4141
Epoch 133/1000
4000/4000 [==============================] - 1s 361us/sample - loss: 44.9794 - mae: 44.9794 - mse: 21186.5645
Epoch 134/1000
4000/4000 [==============================] - 2s 392us/sample - loss: 44.5085 - mae: 44.5085 - mse: 21123.9844
Epoch 135/1000
4000/4000 [==============================] - 1s 335us/sample - loss: 44.8435 - mae: 44.8435 - mse: 21136.6270
Epoch 136/1000
4000/4000 [==============================] - 2s 394us/sample - loss: 44.8971 - mae: 44.8971 - mse: 21158.7227
Epoch 137/1000
4000/4000 [==============================] - 1s 322us/sample - loss: 45.0181 - mae: 45.0181 - mse: 21136.6133
Epoch 138/1000
4000/4000 [==============================] - 1s 346us/sample - loss: 44.7481 - mae: 44.7481 - mse: 21093.5605
Epoch 139/1000


4000/4000 [==============================] - 1s 337us/sample - loss: 44.4404 - mae: 44.4403 - mse: 21072.5508
Epoch 197/1000
4000/4000 [==============================] - 1s 345us/sample - loss: 44.3244 - mae: 44.3244 - mse: 21058.0098
Epoch 198/1000
4000/4000 [==============================] - 1s 359us/sample - loss: 44.5373 - mae: 44.5373 - mse: 21117.9141
Epoch 199/1000
4000/4000 [==============================] - 1s 349us/sample - loss: 44.9139 - mae: 44.9139 - mse: 21207.0586
Epoch 200/1000
4000/4000 [==============================] - 1s 328us/sample - loss: 44.4779 - mae: 44.4779 - mse: 21060.3027
Epoch 201/1000
4000/4000 [==============================] - 1s 368us/sample - loss: 44.7019 - mae: 44.7019 - mse: 21139.8945
Epoch 202/1000
4000/4000 [==============================] - 1s 341us/sample - loss: 44.4563 - mae: 44.4564 - mse: 21075.8867
Epoch 203/1000
4000/4000 [==============================] - 1s 354us/sample - loss: 44.5569 - mae: 44.5569 - mse: 21075.4805
Epoch 204/1000


4000/4000 [==============================] - 2s 388us/sample - loss: 44.9351 - mae: 44.9351 - mse: 21146.5312
Epoch 262/1000
4000/4000 [==============================] - 2s 421us/sample - loss: 44.3916 - mae: 44.3916 - mse: 21013.1953
Epoch 263/1000
4000/4000 [==============================] - 2s 382us/sample - loss: 44.4492 - mae: 44.4492 - mse: 21084.5527
Epoch 264/1000
4000/4000 [==============================] - 2s 402us/sample - loss: 44.4431 - mae: 44.4430 - mse: 21069.3418
Epoch 265/1000
4000/4000 [==============================] - 2s 420us/sample - loss: 44.4753 - mae: 44.4753 - mse: 21097.0664
Epoch 266/1000
4000/4000 [==============================] - 2s 413us/sample - loss: 44.6667 - mae: 44.6667 - mse: 21110.2715
Epoch 267/1000
4000/4000 [==============================] - 2s 399us/sample - loss: 44.6259 - mae: 44.6258 - mse: 21191.7559
Epoch 268/1000
4000/4000 [==============================] - 2s 406us/sample - loss: 44.4004 - mae: 44.4003 - mse: 21094.5781
Epoch 269/1000


...

4000/4000 [==============================] - 2s 416us/sample - loss: 43.1378 - mae: 43.1378 - mse: 20667.1719
Epoch 718/1000
4000/4000 [==============================] - 2s 390us/sample - loss: 43.1848 - mae: 43.1848 - mse: 20615.9863
Epoch 719/1000
4000/4000 [==============================] - 1s 373us/sample - loss: 43.3716 - mae: 43.3715 - mse: 20681.6504
Epoch 720/1000
4000/4000 [==============================] - 1s 374us/sample - loss: 43.2687 - mae: 43.2687 - mse: 20672.1289
Epoch 721/1000
4000/4000 [==============================] - 2s 426us/sample - loss: 43.1025 - mae: 43.1025 - mse: 20670.5781
Epoch 722/1000
4000/4000 [==============================] - 2s 375us/sample - loss: 43.1552 - mae: 43.1552 - mse: 20650.4961
Epoch 723/1000
4000/4000 [==============================] - 2s 391us/sample - loss: 43.0353 - mae: 43.0353 - mse: 20631.7578
Epoch 724/1000
4000/4000 [==============================] - 2s 423us/sample - loss: 43.0742 - mae: 43.0742 - mse: 20637.5508
Epoch 725/1000


4000/4000 [==============================] - 1s 314us/sample - loss: 43.0411 - mae: 43.0411 - mse: 20665.1367
Epoch 783/1000
4000/4000 [==============================] - 1s 304us/sample - loss: 43.1571 - mae: 43.1571 - mse: 20659.0703
Epoch 784/1000
4000/4000 [==============================] - 1s 296us/sample - loss: 43.1357 - mae: 43.1357 - mse: 20680.2520
Epoch 785/1000
4000/4000 [==============================] - 1s 302us/sample - loss: 43.1887 - mae: 43.1887 - mse: 20690.3242
Epoch 786/1000
4000/4000 [==============================] - 1s 298us/sample - loss: 43.2019 - mae: 43.2019 - mse: 20643.8516
Epoch 787/1000
4000/4000 [==============================] - 1s 331us/sample - loss: 43.0362 - mae: 43.0362 - mse: 20605.0371
Epoch 788/1000
4000/4000 [==============================] - 1s 291us/sample - loss: 43.3615 - mae: 43.3615 - mse: 20675.7793
Epoch 789/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 42.9516 - mae: 42.9516 - mse: 20613.1660
Epoch 790/1000


4000/4000 [==============================] - 1s 242us/sample - loss: 43.0692 - mae: 43.0691 - mse: 20650.9375
Epoch 847/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 43.1619 - mae: 43.1619 - mse: 20641.3164
Epoch 848/1000
4000/4000 [==============================] - 1s 228us/sample - loss: 43.0677 - mae: 43.0677 - mse: 20682.9785
Epoch 849/1000
4000/4000 [==============================] - 1s 278us/sample - loss: 43.1118 - mae: 43.1117 - mse: 20638.8223
Epoch 850/1000
4000/4000 [==============================] - 1s 263us/sample - loss: 43.5274 - mae: 43.5274 - mse: 20708.5273
Epoch 851/1000
4000/4000 [==============================] - 1s 236us/sample - loss: 43.0554 - mae: 43.0554 - mse: 20610.4355
Epoch 852/1000
4000/4000 [==============================] - 1s 236us/sample - loss: 43.0692 - mae: 43.0692 - mse: 20638.7949s - loss: 44.9471 - mae: 44.9471 - mse
Epoch 853/1000
4000/4000 [==============================] - 1s 238us/sample - loss: 43.2104 - mae: 43.

4000/4000 [==============================] - 1s 271us/sample - loss: 43.2054 - mae: 43.2054 - mse: 20639.9961
Epoch 910/1000
4000/4000 [==============================] - 1s 243us/sample - loss: 42.9814 - mae: 42.9814 - mse: 20636.8418
Epoch 911/1000
4000/4000 [==============================] - 1s 246us/sample - loss: 42.9705 - mae: 42.9705 - mse: 20621.5938
Epoch 912/1000
4000/4000 [==============================] - 1s 239us/sample - loss: 43.0311 - mae: 43.0311 - mse: 20638.3828
Epoch 913/1000
4000/4000 [==============================] - 1s 235us/sample - loss: 43.4788 - mae: 43.4788 - mse: 20752.7285s - loss: 37.6609 - mae: 37.6609 - mse
Epoch 914/1000
4000/4000 [==============================] - 1s 234us/sample - loss: 42.9292 - mae: 42.9292 - mse: 20657.1855
Epoch 915/1000
4000/4000 [==============================] - 1s 273us/sample - loss: 43.1109 - mae: 43.1109 - mse: 20655.4473
Epoch 916/1000
4000/4000 [==============================] - 1s 252us/sample - loss: 42.8229 - mae: 42.

4000/4000 [==============================] - 1s 230us/sample - loss: 43.1776 - mae: 43.1775 - mse: 20660.3301
Epoch 974/1000
4000/4000 [==============================] - 1s 227us/sample - loss: 43.1301 - mae: 43.1301 - mse: 20612.1133
Epoch 975/1000
4000/4000 [==============================] - 1s 240us/sample - loss: 43.1661 - mae: 43.1661 - mse: 20622.1465
Epoch 976/1000
4000/4000 [==============================] - 1s 290us/sample - loss: 42.8116 - mae: 42.8116 - mse: 20607.3887
Epoch 977/1000
4000/4000 [==============================] - 1s 284us/sample - loss: 43.0045 - mae: 43.0046 - mse: 20631.2891
Epoch 978/1000
4000/4000 [==============================] - 1s 267us/sample - loss: 43.1122 - mae: 43.1122 - mse: 20653.8027
Epoch 979/1000
4000/4000 [==============================] - 1s 301us/sample - loss: 42.9784 - mae: 42.9784 - mse: 20629.7051
Epoch 980/1000
4000/4000 [==============================] - 1s 302us/sample - loss: 43.2466 - mae: 43.2466 - mse: 20697.8867
Epoch 981/1000


In [9]:
print(scores)
print("Mean MAE:", scores.mean())
print("Standard Deviation of the mean:", scores.std())

[-37.32018533 -55.55617752 -34.63468156 -38.08883277 -40.2920885 ]
Mean MAE: -41.17839313697815
Standard Deviation of the mean: 7.412928323345279
